In [1]:
import os
import pyodbc
from sqlalchemy import create_engine
import pandas as pd


In [2]:
#pip install ipython-sql(this enables the use of SQL magic functions that contain % and %% , allowing you to write SQL style code right)
#pip install sqlalchemy(it will mainly be used to store SQL queries into a pandas dataframe.)
#pip install pyodbc (SQL server, you would use  pyodbc:)

# Load the IPython SQL extension.
%load_ext sql

In [3]:
server =os.environ.get('SERVER')
database ='master'

# Connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Connect to the database server
connection = pyodbc.connect(connection_string)
connection.autocommit = True  # Set autocommit mode to True
cursor = connection.cursor()

# Create the database
database_name = 'OnlineRetailDb'
check_database = f"SELECT COUNT(*) FROM sys.databases WHERE name = '{database_name}'"
cursor.execute(check_database)
if cursor.fetchone()[0] == 0:
 # Create the database if it doesn't exist
    create_database = f'CREATE DATABASE {database_name}'
    cursor.execute(create_database)
    print(f"Database '{database_name}' created.")
else:
     print(f"Database '{database_name}' already exists.")
    
create_table_query = '''
    BEGIN TRY
        CREATE TABLE OnlineRetail(
            InvoiceNo INT PRIMARY KEY,
            StockCode VARCHAR(50),
            Description VARCHAR(255),
            Quantity NUMERIC,
            InvoiceDate DATE,
            UnitPrice DECIMAL(18, 2),
            CustomerID INT ,
            Country VARCHAR(255)
      )
    END TRY
    BEGIN CATCH
        IF ERROR_NUMBER() <> 2714
            THROW;
    END CATCH
'''
# Execute the CREATE TABLE statement
cursor.execute(create_table_query)


# Close the cursor and connection
cursor.close()
connection.close()

Database 'OnlineRetailDb' already exists.


In [4]:

# Create the SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc://{server}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')


# Specify the path to the CSV file to be inserted into the database.
csv_file_path = 'G:/My Drive/Analytics/Online Retail.xlsx'

# Read the CSV file into a pandas DataFrame.
df = pd.read_excel(csv_file_path)

# Use the `to_sql()` method of the DataFrame object to insert the data into the database table.
df.to_sql('OnlineRetail', engine, if_exists='append', index=False)

93

In [5]:
%sql mssql+pyodbc://{server}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server



In [6]:
%%sql 
select * from OnlineRetail

 * mssql+pyodbc://FEMUNDO\SQLEXPRESS/OnlineRetailDb?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.
